In [1]:
import re
import nltk
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import warnings
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
warnings.filterwarnings("ignore",category=DeprecationWarning)
%matplotlib inline

#https://na80.lightning.force.com/lightning/r/Report/00O1P000003mYjbUAE/view

/Users/emeryames/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
df = pd.read_csv('august_data.csv')
df.Description.head()

0    Chat transcript\n\n\t\t\tE-mail: bistro@tohu.c...
1    Chat transcript\n\n\t\t\tName: Rebekkah Damian...
2    Chat transcript\n\n\t\t\tE-mail: jaycen@me.com...
3    Chat transcript\n\n\t\t\tE-mail: mario@elevati...
4    Chat transcript\n\n\t\t\tE-mail: opnaples@came...
Name: Description, dtype: object

In [3]:
data = df.Description.values.tolist()
data = [re.sub('\S*@\S*\s?', "", sent) for sent in data]
data = [re.sub('\s+', " ", sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]
data = [re.sub('https:\S+',"", sent) for sent in data]

pprint(data[:1])

['Chat transcript E-mail: Victoria Fri, 08/27/21 12:54:54 pm America/Toronto '
 'BevSpot Support here! How can we help? Bistro Tohu 12:55:16 pm hi i would '
 'like to import invoices can you activate the addon? Victoria 12:56:14 pm '
 'Hello - Thank you for reaching out! Are you referring to invoice processing? '
 'Bistro Tohu 12:56:25 pm i guess i want to import csv and pdf for creating '
 'items Victoria 01:02:13 pm Ok... Just to clarify, these are going to be '
 'invoiced from a vendor, and thats what youre looking to enter into the '
 'account? Or do you have a list of items on a file that we can upload through '
 'the items section? Just double-checking because currently, invoice '
 'processing is an add-on feature that is not included within the freemium '
 'program. Bistro Tohu 01:02:55 pm from invoice i m ok to pay Victoria '
 '01:04:12 pm Ok, thank you for this information. Hang tight for a quick '
 'moment while I reach out to your account manager. Bistro Tohu 01:04:52 pm '
 

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data_words = list(sent_to_words(data))

In [5]:
bigram = gensim.models.Phrases(data, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [6]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
data_words_nostops = remove_stopwords(data)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [8]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

In [9]:
#no touch
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [10]:
#score
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.176994217240875

Coherence Score:  0.4154599827415786


In [11]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12    -0.009735  0.114487       1        1  21.754835
1      0.208548 -0.028416       2        1  11.892785
8      0.197799  0.138925       3        1  11.411606
6      0.069338 -0.289839       4        1  10.046632
5      0.005634  0.207259       5        1   9.356603
17     0.061578  0.236483       6        1   5.130808
14     0.172180  0.047828       7        1   4.946258
2     -0.186099  0.131080       8        1   4.327380
18     0.099998  0.096808       9        1   4.258631
13    -0.213825 -0.118307      10        1   3.883007
16    -0.242498  0.058280      11        1   3.784207
3      0.156563 -0.141165      12        1   3.667758
0     -0.106181  0.146090      13        1   3.532303
9      0.063928 -0.077411      14        1   1.246011
11    -0.016969 -0.024992      15        1   0.645314
10    -0.050598 -0.096437      16        1   0.024888
7     -0.052387 -0.100745      17        1   0.023044
15    -0.052093 -0.099871      18        1   0.022665
19    -0.052497 -0.099741      19        1   0.022635
4     -0.052682 -0.100317      20        1   0.022629, topic_info=        Term       Freq      Total Category  logprob  loglift
56     order  28.000000  28.000000  Default  30.0000  30.0000
45   invoice  40.000000  40.000000  Default  29.0000  29.0000
12      chat  92.000000  92.000000  Default  28.0000  28.0000
317    erick  39.000000  39.000000  Default  27.0000  27.0000
59        pm  98.000000  98.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
38      help   0.001204  51.042249  Topic20  -6.3836  -2.2614
66    report   0.001199  35.877940  Topic20  -6.3871  -1.9124
2    account   0.001196  42.106825  Topic20  -6.3896  -2.0749
9    bevspot   0.001195  49.748461  Topic20  -6.3904  -2.2425
144    today   0.001192  17.697703  Topic20  -6.3933  -1.2119

[1274 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
161       8  0.701717           _
0         1  0.132858        able
0         3  0.265717        able
0         4  0.199288        able
0         5  0.132858        able
...     ...       ...         ...
148       9  0.178940        work
148      10  0.178940        work
550       5  0.667786  workaround
609       4  0.809889       worth
354       4  0.649079       wrong

[1324 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 2, 9, 7, 6, 18, 15, 3, 19, 14, 17, 4, 1, 10, 12, 11, 8, 16, 20, 5])